In [1]:
import numpy as np
import pandas as pd
from minepy import MINE
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.io import arff
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import TomekLinks
from scipy.io import arff
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline
import fileinput
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Bidirectional
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

train_df = pd.read_csv("dataset/KDDTrain+.txt", header=None, names=columns)
test_df = pd.read_csv("dataset/KDDTest+.txt", header=None, names=columns)
df = pd.concat([train_df, test_df])
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [3]:
#Shape of dataset
print("Shape of DF:",df.shape)

Shape of DF: (148517, 43)


In [4]:
df['attack'].value_counts()

attack
normal             77054
neptune            45871
satan               4368
ipsweep             3740
smurf               3311
portsweep           3088
nmap                1566
back                1315
guess_passwd        1284
mscan                996
warezmaster          964
teardrop             904
warezclient          890
apache2              737
processtable         685
snmpguess            331
saint                319
mailbomb             293
pod                  242
snmpgetattack        178
httptunnel           133
buffer_overflow       50
land                  25
multihop              25
rootkit               23
named                 17
ps                    15
sendmail              14
xterm                 13
imap                  12
loadmodule            11
ftp_write             11
xlock                  9
phf                    6
perl                   5
xsnoop                 4
spy                    2
worm                   2
sqlattack              2
udpstorm          

In [5]:
#Checking for null values/missing values
print("Sum of null values in DF:",df.isna().sum().sum())

Sum of null values in DF: 0


In [6]:
#Remove duplicates
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(148517, 43)
(147907, 43)


In [7]:
train_df['attack'].describe()

count     125973
unique        23
top       normal
freq       67343
Name: attack, dtype: object

In [8]:
# #Variables Balance
# attack_categories = {
#     'normal': 'normal',
#     'neptune': 'DoS',
#     'back': 'DoS',
#     'land': 'DoS',
#     'pod': 'DoS',
#     'smurf': 'DoS',
#     'teardrop': 'DoS',
#     'mailbomb': 'DoS',
#     'apache2': 'DoS',
#     'processtable': 'DoS',
#     'udpstorm': 'DoS',
#     'worm': 'DoS',
#     'buffer_overflow': 'U2R',
#     'loadmodule': 'U2R',
#     'perl': 'U2R',
#     'rootkit': 'U2R',
#     'ps': 'U2R',
#     'sqlattack': 'U2R',
#     'xterm': 'U2R',
#     'ftp_write': 'R2L',
#     'guess_passwd': 'R2L',
#     'imap': 'R2L',
#     'multihop': 'R2L',
#     'phf': 'R2L',
#     'spy': 'R2L',
#     'warezclient': 'R2L',
#     'warezmaster': 'R2L',
#     'sendmail': 'R2L',
#     'named': 'R2L',
#     'snmpgetattack': 'R2L',
#     'snmpguess': 'R2L',
#     'xlock': 'R2L',
#     'xsnoop': 'R2L',
#     'httptunnel': 'R2L',
#     'ipsweep': 'Probe',
#     'nmap': 'Probe',
#     'portsweep': 'Probe',
#     'satan': 'Probe',
#     'mscan': 'Probe',
#     'saint': 'Probe'
# }

# df['attack'] = df['attack'].map(attack_categories)

attack = df.attack.map(lambda a: 0 if a == 'normal' else 1)

df['attack'] = attack

In [9]:
catData = df.select_dtypes(['object']).columns
catData

Index(['protocol_type', 'service', 'flag'], dtype='object')

In [10]:
# Initialize LabelEncoder 
label_encoders = {}
for col in catData:
    # Initialize LabelEncoder for each column
    label_encoders[col] = LabelEncoder()
    
    # Fit and transform the column
    df[col] = label_encoders[col].fit_transform(df[col])
    
    # Get the label encoding data
    print(f"Column: {col}")
    print(f"Encoded labels: {label_encoders[col].classes_}")
    print(f"Encoding mapping: {dict(zip(label_encoders[col].classes_, range(len(label_encoders[col].classes_))))}")
    print()



Column: protocol_type
Encoded labels: ['icmp' 'tcp' 'udp']
Encoding mapping: {'icmp': 0, 'tcp': 1, 'udp': 2}

Column: service
Encoded labels: ['IRC' 'X11' 'Z39_50' 'aol' 'auth' 'bgp' 'courier' 'csnet_ns' 'ctf'
 'daytime' 'discard' 'domain' 'domain_u' 'echo' 'eco_i' 'ecr_i' 'efs'
 'exec' 'finger' 'ftp' 'ftp_data' 'gopher' 'harvest' 'hostnames' 'http'
 'http_2784' 'http_443' 'http_8001' 'imap4' 'iso_tsap' 'klogin' 'kshell'
 'ldap' 'link' 'login' 'mtp' 'name' 'netbios_dgm' 'netbios_ns'
 'netbios_ssn' 'netstat' 'nnsp' 'nntp' 'ntp_u' 'other' 'pm_dump' 'pop_2'
 'pop_3' 'printer' 'private' 'red_i' 'remote_job' 'rje' 'shell' 'smtp'
 'sql_net' 'ssh' 'sunrpc' 'supdup' 'systat' 'telnet' 'tftp_u' 'tim_i'
 'time' 'urh_i' 'urp_i' 'uucp' 'uucp_path' 'vmnet' 'whois']
Encoding mapping: {'IRC': 0, 'X11': 1, 'Z39_50': 2, 'aol': 3, 'auth': 4, 'bgp': 5, 'courier': 6, 'csnet_ns': 7, 'ctf': 8, 'daytime': 9, 'discard': 10, 'domain': 11, 'domain_u': 12, 'echo': 13, 'eco_i': 14, 'ecr_i': 15, 'efs': 16, 'exec': 

In [11]:
# Initialize ADASYN for oversampling
adasyn = ADASYN()
# Initialize TomekLinks for undersampling
tomek_links = TomekLinks()

In [12]:
X = df.drop(columns=['attack'])  # Takking 'attack' as target variable
y = df['attack'] 

In [13]:
print("Intial Y value count:",y.value_counts())

Intial Y value count: attack
0    76967
1    70940
Name: count, dtype: int64


In [14]:
# Oversample the DataFrame
X_resampled, y_resampled = adasyn.fit_resample(X, y) 

  File "c:\Users\ankit\miniconda3\envs\myenv\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\ankit\miniconda3\envs\myenv\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\ankit\miniconda3\envs\myenv\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\ankit\miniconda3\envs\myenv\lib\subprocess.py", line 1436, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


In [15]:
y_resampled.value_counts()

attack
0    76967
1    76955
Name: count, dtype: int64

In [16]:
y

0        0
1        0
2        1
3        0
4        0
        ..
22537    1
22539    0
22540    0
22541    1
22543    1
Name: attack, Length: 147907, dtype: int64

In [17]:
# 1. MIC Feature Selection

def calculate_mic(X, y):
    mine = MINE()
    mic_scores = []
    for column in X.columns:
        mine.compute_score(X[column], y)
        mic_scores.append(mine.mic())
    return mic_scores

In [18]:
# Calculate MIC scores
mic_scores = calculate_mic(X_resampled, y_resampled)

In [19]:
kernal_evals = dict()
def evaluate_classification(model, name, X_train, X_test, y_train, y_test):
    train_accuracy = metrics.accuracy_score(y_train, model.predict(X_train))
    test_accuracy = metrics.accuracy_score(y_test, model.predict(X_test))
    
    train_precision = metrics.precision_score(y_train, model.predict(X_train))
    test_precision = metrics.precision_score(y_test, model.predict(X_test))
    
    train_recall = metrics.recall_score(y_train, model.predict(X_train))
    test_recall = metrics.recall_score(y_test, model.predict(X_test))
    
    kernal_evals[str(name)] = [train_accuracy, test_accuracy, train_precision, test_precision, train_recall, test_recall]
    print("Training Accuracy " + str(name) + " {}  Test Accuracy ".format(train_accuracy*100) + str(name) + " {}".format(test_accuracy*100))
    print("Training Precesion " + str(name) + " {}  Test Precesion ".format(train_precision*100) + str(name) + " {}".format(test_precision*100))
    print("Training Recall " + str(name) + " {}  Test Recall ".format(train_recall*100) + str(name) + " {}".format(test_recall*100))
    
    actual = y_test
    predicted = model.predict(X_test)
    confusion_matrix = metrics.confusion_matrix(actual, predicted)
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['normal', 'attack'])

    fig, ax = plt.subplots(figsize=(10,10))
    ax.grid(False)
    cm_display.plot(ax=ax)


In [21]:

# Create a DataFrame with MIC scores and rankings
mic_df = pd.DataFrame({
    'Feature': X_resampled.columns,
    'MIC Score': mic_scores
})
mic_df['Rank'] = mic_df['MIC Score'].rank(ascending=False)
mic_df = mic_df.sort_values('Rank')

print(mic_df)

                        Feature  MIC Score  Rank
4                     src_bytes   0.735555   1.0
2                       service   0.601555   2.0
5                     dst_bytes   0.578064   3.0
3                          flag   0.443519   4.0
32           dst_host_srv_count   0.427584   5.0
29                diff_srv_rate   0.426152   6.0
28                same_srv_rate   0.414640   7.0
33       dst_host_same_srv_rate   0.404117   8.0
34       dst_host_diff_srv_rate   0.381630   9.0
11                    logged_in   0.336309  10.0
37         dst_host_serror_rate   0.327300  11.0
38     dst_host_srv_serror_rate   0.316199  12.0
24                  serror_rate   0.311717  13.0
22                        count   0.302264  14.0
25              srv_serror_rate   0.297288  15.0
36  dst_host_srv_diff_host_rate   0.248233  16.0
41                        level   0.240991  17.0
35  dst_host_same_src_port_rate   0.195900  18.0
31               dst_host_count   0.150086  19.0
30           srv_dif

In [22]:
# Select top features 
top_features = mic_df['Feature'][:20].tolist()
X_selected = X_resampled[top_features]

In [23]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size=0.2, random_state=42)


In [24]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for CNN and RNN
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))


In [25]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available())

True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [26]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:

# Function to create and train models
def create_and_train_model(model_func, X_train, y_train, model_name):
    with tf.device('/GPU:0'):
        model = model_func(X_train.shape[1])
        history = model.fit(
            X_train, y_train, 
            epochs=5, 
            batch_size=32, 
            validation_split=0.2,
            verbose=1
        )
    print(f"{model_name} training completed.")
    return model, history

In [28]:
# 1. Multi-layer Perceptron (MLP)
def create_mlp_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

mlp_model, mlp_history = create_and_train_model(create_mlp_model, X_train_scaled, y_train, "MLP")


Epoch 1/5
3079/3079 [==============================] - 10s 3ms/step - loss: 0.0437 - accuracy: 0.9858 - val_loss: 0.0275 - val_accuracy: 0.9909
Epoch 2/5
3079/3079 [==============================] - 9s 3ms/step - loss: 0.0211 - accuracy: 0.9924 - val_loss: 0.0186 - val_accuracy: 0.9937
Epoch 3/5
3079/3079 [==============================] - 10s 3ms/step - loss: 0.0179 - accuracy: 0.9937 - val_loss: 0.0181 - val_accuracy: 0.9936
Epoch 4/5
3079/3079 [==============================] - 10s 3ms/step - loss: 0.0161 - accuracy: 0.9943 - val_loss: 0.0185 - val_accuracy: 0.9939
Epoch 5/5
3079/3079 [==============================] - 10s 3ms/step - loss: 0.0146 - accuracy: 0.9947 - val_loss: 0.0138 - val_accuracy: 0.9952
MLP training completed.


In [29]:
# Evaluate models
def evaluate_model(model, X_test, y_test, model_name):
    with tf.device('/GPU:0'):
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"{model_name} - Test accuracy: {accuracy:.4f}")

evaluate_model(mlp_model, X_test_scaled, y_test, "MLP")

MLP - Test accuracy: 0.9952


In [30]:
mlp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1344      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,969
Trainable params: 3,969
Non-trainable params: 0
_________________________________________________________________
